In [ ]:
# umbridge server
import umbridge
from scipy.integrate import solve_ivp 
import nest_asyncio
nest_asyncio.apply()
import numpy as np


class PDEModel(umbridge.Model):
    def __init__(self):
        super().__init__("MMC_Solver") #Give a name to the model
        # Define transformation matrix T which decouples the equations
        self.T = np.array(  [[1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6)],
                            [-1/np.sqrt(6), -1/np.sqrt(6), -1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6)],
                            [1/2, -1/2, 0, -1/2, 1/2, 0],
                            [1/np.sqrt(12), 1/np.sqrt(12), -1/np.sqrt(3), -1/np.sqrt(12), -1/np.sqrt(12), 1/np.sqrt(3)],
                            [-1/2, 1/2, 0, -1/2, 1/2, 0],
                            [-1/np.sqrt(12), -1/np.sqrt(12), 1/np.sqrt(3), -1/np.sqrt(12), -1/np.sqrt(12), 1/np.sqrt(3)]]
                         )

        # Define C
        self.C_tilde= np.diag([2, 0, 0, 0, 2, 2])

        # Define size of the square matrices (e.g. A_tilde)
        self.n = 6

        #Define time interval in which the ODE will be solved numerically as well as the time resolution all in seconds (s)
        self.t_start,self.t_stop,self.t_step = 0, 20e-3, 125e-6
        self.t_eval = np.arange(self.t_start,self.t_stop+self.t_step,self.t_step)
        self.t_span = [self.t_eval[0],self.t_eval[-1]]

    """
    Define the transformed matrices that decouple the equations describing the MMC as demonstrated in doi:10.1109/OJPEL.2021.3065115.
    For clarity, they will all be defined in the following with a "_tilde" ending.
    """

    # A_tilde matrix
    def a_tilde(self, R_arm, R_DC, L_arm, L_DC, R_AC, L_AC, config):
        A = np.zeros((self.n, self.n))
        np.fill_diagonal(A, [0, -(R_arm+3*R_DC)/(L_arm+3*L_DC), -(R_arm)/(L_arm), -(R_arm)/(L_arm), -(R_arm+2*R_AC)/(L_arm+2*L_AC), -(R_arm+2*R_AC)/(L_arm+2*L_AC)])
        return A  

    # B_tilde matrix
    def b_tilde(self, L_arm, L_DC, L_AC, config):
        B = np.zeros((self.n, self.n))
        np.fill_diagonal(B, [0, -(1)/(L_arm+3*L_DC), -(1)/(L_arm), -(1)/(L_arm), -(1)/(L_arm+2*L_AC), -(1)/(L_arm+2*L_AC)])
        return B  

    # F_tilde matrix
    def f_tilde(self, L_arm, L_DC, L_AC,config):
        F = np.zeros((self.n , self.n-2))
        F[1][3] = (-(np.sqrt(6)))/(2*L_arm+6*L_DC)
        F[4][0] = (1/(L_arm+2*L_AC))
        F[4][1] = (-1/(L_arm+2*L_AC))
        F[5][0] = (np.sqrt(3)/(3*L_arm+6*L_AC))
        F[5][1] = (np.sqrt(3)/(3*L_arm+6*L_AC))
        F[5][2] = (-(2*np.sqrt(3))/(3*L_arm+6*L_AC))
        return F  

    # Disturbance vector z
    def disturbance_vector(self, t, omega_AC, U_AC, U_DC, config):
        z = []
        for k in range(3):
            z.append(U_AC * np.cos (omega_AC * t - (2 * np.pi * (k-1))/3))

        z.append(U_DC)
        return np.array(z)
    
    # Define function to convert transformed arm currents x_tilde to output y
    def get_mmc_output(self, x_tilde):
        y_1 = -x_tilde[4]-1/np.sqrt(3)*x_tilde[5]
        y_2 = x_tilde[4]-1/np.sqrt(3)*x_tilde[5]
        y_3 = 2/np.sqrt(3)*x_tilde[5]
        return np.array([y_1,
                         y_2,
                         y_3])
    
    # Define ansatz for control vector u assuming it to display oscillatory behaviour with the same angular frequency omega_AC
    def control_vector(self, t, omega_AC, p, config):
        u1 = p[0] + p[1] * np.sin(omega_AC * t + p[2]) + p[3] * np.cos(omega_AC * t + p[4])
        u2 = p[5] + p[6] * np.sin(omega_AC * t + p[7]) + p[8] * np.cos(omega_AC * t + p[9])
        u3 = p[10] + p[11] * np.sin(omega_AC * t + p[12]) + p[13] * np.cos(omega_AC * t + p[14])
        u4 = p[15] + p[16] * np.sin(omega_AC * t + p[17]) + p[18] * np.cos(omega_AC * t + p[19])
        u5 = p[20] + p[21] * np.sin(omega_AC * t + p[22]) + p[23] * np.cos(omega_AC * t + p[24])
        u6 = p[25] + p[26] * np.sin(omega_AC * t + p[27]) + p[28] * np.cos(omega_AC * t + p[29])

        return np.array([u1, u2, u3, u4, u5, u6])   
    
    # Define the ODE that describes the MMC
    def MMC_ODE(self,t, x_tilde, p, A_tilde, B_tilde, F_tilde, T, omega_AC, U_AC, U_DC, config):
        u = self.control_vector(t,omega_AC,p,config)
        z = self.disturbance_vector(t,omega_AC,U_AC,U_DC,config)
        dx_tildedt= A_tilde @ x_tilde + B_tilde @ (T @ u) + F_tilde @ z

        return dx_tildedt
    
    # Define function to determine arm powers for the numerically computed values of x_tilde_1..x_tilde_6 at the same t_eval points
    # It is important to clarify that x_tilde_power=x_power, i.e. the values are the same in the transformed and untransformed (physically)
    # meaningful system
    def get_arm_powers(self, x_tilde, omega_AC,p, config):
        u_t = np.array([self.control_vector(t,omega_AC,p,config) for t in self.t_eval]) # shape = (161,6)
        x_tilde_power = np.multiply(self.T.T @ x_tilde,self.T.T @ (self.T @ u_t.T)) # shape = (6,161)

        return x_tilde_power

    # Define function to calculate arm energies using Euler's method for numerical integration 
    def get_arm_energies(self, x_tilde_1_6, u):
        # Initialize the output array for x_{7-12}
        n = x_tilde_1_6.shape[1]  # n should be 161
        x_7_12 = np.zeros((6, n))  # Assuming 6 output components

        # Compute the time step size
        dt = self.t_step 

        #Compute u_tilde
        u_tilde = self.T @ u.T

        # Compute x_{7-12} at each time step
        for t in range(n):
            # Compute dot product (Hadamard product)
            dot_x_7_12 = (self.T.T @ u_tilde[:, t]) * (self.T.T @ u_tilde[:, t])  # Element-wise multiplication

            # Integrate using Euler's method
            if t == 0:
                x_7_12[:, t] = 0  # Initial condition, modify as needed
            else:
                x_7_12[:, t] = x_7_12[:, t-1] + dot_x_7_12 * dt  # Euler's integration step

        return x_7_12

    
    # Define list of inputs. They correspond to:
    # [0]: Values of components like R_arm, L_arm,.. which will later be varied within a range of +-10% in a UQ analysis
    # [1]: List of 30 parameters that 'control_vector' needs as input 
    # [2]: List of quantities that are taken as constant like the voltages of the DC and AC grid
    # [3]: Initial values for arm currents in transformed system, i.e. x_tilde0
    def get_input_sizes(self, config):
        return [6, 30, 3, 6]   
    
    # Define output size. It returns:
    # [0]: The output of the MMC, shape = (3,161)
    # [1]: The control vector u, shape = (6,161)
    # [2]: The arm powers, shape = (6,161)
    # [3]: The arm energies, shape = (6,161)
    def get_output_sizes(self, config):
        return [1,1,1,1]
        
    def __call__(self, parameters, config):
        """
        For future debugging. If JSONDecodeError arises check if the response can be converted to json format (e.g. not possible for 'np.float64' or 'array')
        response = requests.post(f"{url}/Evaluate", json=inputParams)
        print("Response Status Code:", response.status_code)
        print("Response Text:", response.text)  # Check the raw response content
        """
        R_arm = parameters [0][0]
        R_DC = parameters [0][1]
        R_AC = parameters [0][2]
        L_arm = parameters [0][3]
        L_DC = parameters [0][4]
        L_AC = parameters [0][5]
        p = parameters[1]
        omega_AC = parameters[2][0]
        U_AC = parameters[2][1]
        U_DC = parameters[2][2]
        x_tilde0 = parameters[3]
        A_tilde = self.a_tilde(R_arm, R_DC, L_arm, L_DC, R_AC, L_AC, config)
        B_tilde = self.b_tilde(L_arm, L_DC, L_AC, config)
        F_tilde = self.f_tilde(L_arm, L_DC, L_AC, config)

        # Solve ODE -> shape = (6,161)
        sol = solve_ivp(self.MMC_ODE, t_span=self.t_span, y0=x_tilde0, method="RK45", t_eval=self.t_eval, args=(p,A_tilde,B_tilde,F_tilde,self.T,omega_AC,U_AC,U_DC, config))
        x_tilde_sol = sol.y
        y_sol = self.get_mmc_output(x_tilde_sol)

        # Calculate control vector with the given set of parameters -> shape = (6,161)
        u_p = np.array([self.control_vector(t,omega_AC,p=p,config=config) for t in self.t_eval])

        # Calculate arm powers
        arm_powers = self.get_arm_powers(x_tilde_sol,omega_AC=omega_AC,p=p,config=config) 

        #Calculate arm energies
        arm_energies = self.get_arm_energies(x_tilde_sol,u_p)

        #IMPORTANT: convert np.ndarrays to lists otherwise there will appear a JSONDecodeError because an Numpy Array represented as an 'array' is not valid Json!
        return [[y_sol.tolist()],[u_p.tolist()],[arm_powers.tolist()],[arm_energies.tolist()]]  
 
    def supports_evaluate(self):
        return True

pde_model = PDEModel()
umbridge.serve_models([pde_model], 4242) # start model server

======== Running on http://0.0.0.0:4242 ========
(Press CTRL+C to quit)
